# 1-round SPECK 32/32 (ResNet)

## Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pipeline import *

In [ ]:
from dataset_rr.make_train_data import make_train_data
from dataset_rr.speck import Speck

## Importing the dataset

In [ ]:
n_train_samples = 10**6
n_eval_samples = 10**5
n_rounds = 1

cipher = Speck(n_rounds=n_rounds)

key = cipher.draw_keys(1)

In [ ]:
train_samples, train_labels = make_train_data(n_train_samples, cipher, key)
test_samples, test_labels = make_train_data(n_eval_samples, cipher, key)

In [ ]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

## Creating the model

In [ ]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [ ]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])

# units per hidden layer
units = dim*16

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=1000,
    decay_rate=0.01)
learning_rate = 0.1

optimizer = Adam(learning_rate=0.001)
metrics = ['accuracy', 'binary_accuracy']
epochs = 25
batch_size = 5000

### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [ ]:
inputs = Input(shape=input_shape)
net = inputs

for _ in range(8):
    x1 = BatchNormalization()(inputs)
    x1 = Dense(units=units, activation='relu')(net)

    x2 = BatchNormalization()(x1)
    x2 = Dense(units=units, activation='relu')(inputs)

    net = Add()([x1, x2])
    
net = Dense(units=dim, activation='softmax')(net)

neural_network = Model(inputs, net)

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [ ]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

In [ ]:
h_loss = history.history['loss']
h_val_loss = history.history['val_loss']
h_val_accuracy = history.history['val_accuracy']

print(h_loss)
print(h_val_loss)
print(h_val_accuracy)

### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

In [ ]:
results = neural_network.evaluate(test_samples, test_labels, batch_size=batch_size)
print("Test loss: {}".format(results[0]))
print("Test accuracy: {}".format(results[1]))